In [2]:
import pandas as pd
import numpy as np
import gamspy as gp 
import gamspy.math as gpm

## Set Definition

In [ ]:
m = gp.Container()

# PLACEHOLDERS: Replace with actual sets once data is cleaned and the total number of recipes and ingredients are known
I = gp.Set(m, 'I', description="A set containing all of the valid ingredients")
R = gp.Set(m, 'R', description="A set containing all of the recipes that can be made")

## Parameters

In [ ]:
C = gp.Parameter(m, 'C', domain=[I], description="Cost per unit of each of the ingredients")
A = gp.Parameter(m, 'A', domain=[I, R], description="Amount of ingredient i required in recipe r")
B = gp.Parameter(m, 'B', description="Budget to purchase ingredients")
a = gp.Parameter(m, 'a', description="Trade off parameter used to weight the regularizer")

## Variables

In [ ]:
z = gp.Variable(m, 'z', "free", domain=[I], description="Amount of ingredient i purchased")
x = gp.Variable(m, 'x', "binary", domain=[R], description="Indicator variable to make recipe r")
l = gp.Variable(m, 'l', "free", domain=[I], description="Leftover ingredients after all recipes are made")

## Equations

In [ ]:
budget = gp.Equation(m, 'budget', 'regular',
                        description="Constrains the total money spent on ingredients to be within the budget")
budget[:] = gp.Sum(I, C[I] * z[I]) <= B

ingredient_amounts = gp.Equation(m, 'ingredient_amounts', 'regular', domain=[I],
                              description="Ensures enough ingredients are purchased to satisfy the selected recipes")
ingredient_amounts[I] = gp.Sum(R, A[I,R] * x[R]) <= z[I]

waste = gp.Equation(m, 'waste', 'regular', domain=[I],
                          description="Sets the leftover variable equal to the amount of unused ingredients")
waste[I] = l[I] == z[I] -  gp.Sum(R, A[I,R] * x[R])

tot_recipes = gp.Equation(m, 'tot_recipes', description="Ensures enough recipes for a weeks worth of dinner can be made")
tot_recipes[:] = gp.Sum(R, x[R]) >= 7

## Model

In [ ]:
objective = gp.Sum(R, x[R]) - a * gp.Sum(I, gpm.sqr(l))

model = gp.Model(
    container=m,
    name='model',
    problem=gp.Problem.MIP,
    equations=m.getEquations(),
    sense=gp.Sense.MAX,
    objective=objective
)